In [1]:
using Printf, CSV, DataFrames, Statistics, Glob, XLSX, Dates, Plots, TimeSeries, JSON, Distributions, ExpectationMaximization

In [2]:
data_directory = joinpath("/projects", "emco4286", "data", "gads", "trajectories", "ct")
files = glob("gen_*_class_CT_rating_*_state_Texas.csv", data_directory)

64-element Vector{String}:
 "/projects/emco4286/data/gads/tr" ⋯ 26 bytes ⋯ "ass_CT_rating_1_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 26 bytes ⋯ "ass_CT_rating_1_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 26 bytes ⋯ "ass_CT_rating_1_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 26 bytes ⋯ "ass_CT_rating_1_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 26 bytes ⋯ "ass_CT_rating_2_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 26 bytes ⋯ "ass_CT_rating_2_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 26 bytes ⋯ "ass_CT_rating_1_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 26 bytes ⋯ "ass_CT_rating_1_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 26 bytes ⋯ "ass_CT_rating_1_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 26 bytes ⋯ "ass_CT_rating_2_state_Texas.csv"
 ⋮
 "/projects/emco4286/data/gads/tr" ⋯ 26 bytes ⋯ "ass_CT_rating_1_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 26 bytes ⋯ "ass_CT_rating_1_sta

In [3]:
f = files[1]

df = DataFrame(CSV.File(f))

Row,x,y1,y2,y3,y9,ERCOT,y4,Pcp,Tmax,Tmin,y6,y7,y8
,DateTime,Float64,Float64,Int64,Float64,Float64,Int64,Float64?,Float64?,Float64?,Int64?,Missing,Int64
1,2013-01-01T06:00:00,1.0,1.0,2922,1.0,27836.8,1,2.46,59.4,36.4,missing,missing,1
2,2013-01-01T07:00:00,1.0,1.0,2922,1.0,28722.0,1,2.46,59.4,36.4,missing,missing,1
3,2013-01-01T08:00:00,1.0,1.0,2922,1.0,29391.7,1,2.46,59.4,36.4,missing,missing,1
4,2013-01-01T09:00:00,1.0,1.0,2922,1.0,30293.8,1,2.46,59.4,36.4,missing,missing,1
5,2013-01-01T10:00:00,1.0,1.0,2922,1.0,31882.3,1,2.46,59.4,36.4,missing,missing,1
6,2013-01-01T11:00:00,1.0,1.0,2922,1.0,33359.5,1,2.46,59.4,36.4,missing,missing,1
7,2013-01-01T12:00:00,1.0,1.0,2923,1.0,34186.8,1,2.46,59.4,36.4,missing,missing,1
8,2013-01-01T13:00:00,1.0,1.0,2923,1.0,34155.1,1,2.46,59.4,36.4,missing,missing,1
9,2013-01-01T14:00:00,1.0,1.0,2923,1.0,33781.1,1,2.46,59.4,36.4,missing,missing,1


In [4]:
size(df)

(96403, 13)

In [13]:
outage_start = findall(diff(df[!, :y1]) .== -1)
outage_end = findall(diff(df[!, :y1]) .== 1)
ttr_values = outage_end .- outage_start
ttf_values_all_types = outage_start[begin + 1: end] - outage_end[begin:end - 1]
outage_types = df[outage_start .+ 1, :y2]

41-element Vector{Float64}:
 3.0
 3.0
 2.0
 3.0
 2.0
 2.0
 3.0
 2.0
 3.0
 2.0
 ⋮
 2.0
 2.0
 2.0
 3.0
 3.0
 2.0
 2.0
 2.0
 3.0

In [18]:
filter(x -> x > 2, outage_types)

20-element Vector{Float64}:
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0

In [12]:
fo_idx = findall(outage_types .== 2)
ttf_values_fo = ttf_values_all_types[fo_idx]
po_idx = findall(outage_types .== 3)
ttf_values_po = ttf_values_all_types[po_idx[begin: end - 1]]

19-element Vector{Int64}:
 4742
 1842
  688
 1796
  593
 3400
 2760
 2800
  594
  118
    7
 8607
 2402
 1868
 2750
 5611
  419
 9299
   16

In [10]:
ttf_values_all_types

40-element Vector{Int64}:
 4742
 1842
 1073
  688
 4150
  472
 1796
 1255
  593
 3334
    ⋮
  419
   20
 1386
 2154
 9299
   16
  187
 1703
 2342

In [ ]:

os_fo = outage_start[fo_idx]
oe_fo = outage_end[fo_idx]
ttf_values_fo = os_fo[begin + 1: end] - oe_fo[begin:end - 1]
ttr_values_fo = ttr_values[fo_idx]

In [39]:
ph = 8760
foh = 741.35
num_fo = 6.14
poh = 769.13
num_po = 1.08
moh = 291.4
num_mo = 2.04

fo_rate = (ph - moh - poh)/num_fo
po_rate = (ph - moh - foh)/num_po
mo_rate = (ph - foh - poh)/num_mo

3553.6862745098038

In [44]:
mix_guess = MixtureModel([Exponential(fo_rate), Exponential(po_rate), Exponential(mo_rate)], [1/3, 1/3, 1/3])
mix_mle, info = fit_mle(mix_guess, ttf_values_all_types, infos = true)

(MixtureModel{Exponential{Float64}}(K = 3)
components[1] (prior = 0.3500): Exponential{Float64}(θ=1423.5936255732252)
components[2] (prior = 0.2781): Exponential{Float64}(θ=2704.7592348303515)
components[3] (prior = 0.3719): Exponential{Float64}(θ=2659.1454532203497)
, Dict{String, Any}("iterations" => 14, "converged" => true, "logtots" => [-348.72542179932344, -348.6454860430778, -348.6061198950445, -348.5842399343577, -348.5709779478319, -348.5623947162555, -348.5565444695478, -348.5523846384287, -348.5493193948287, -348.54698989686057, -348.5451704268922, -348.54371370748987, -348.54252058996894, -348.5415225006483]))

In [33]:
po_idx = findall(outage_types .== 3)
os_po = outage_start[po_idx]
oe_po = outage_end[po_idx]
ttf_values_po = os_po[begin + 1: end] - oe_po[begin:end - 1]
ttr_values_po = ttr_values[po_idx]

20-element Vector{Int64}:
 468
 265
 234
 111
 345
 127
  12
 106
  14
 187
  15
 321
 123
 128
  90
 122
  14
 133
 133
  16

In [34]:
mo_idx = findall(outage_types .== 4)
os_mo = outage_start[mo_idx]
oe_mo = outage_end[mo_idx]
ttf_values_mo = os_mo[begin + 1: end] - oe_mo[begin:end - 1]
ttr_values_mo = ttr_values[mo_idx]

Int64[]